# What's cooking?

Tensorflow MLPClassifier com camada de Embedding

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
train_df = pd.read_json('./train.json')
test_df = pd.read_json('./test.json')

train_df.sample(5)
# coletando ingredientes:
ingredients = train_df.ingredients.str.join(' ')
# ingredients
# removendo caracteres especiais, stemming e lemmatizing:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in word_tokenize(text)]

train = train_df.copy()
train['X'] = train_df.ingredients.str.join(' ')
train['X'].apply(lemmatize_text)

train.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,cuisine,ingredients,X
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuce black olives grape tomatoes ga...
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",plain flour ground pepper salt tomatoes ground...
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",eggs pepper salt mayonaise cooking oil green c...
3,22213,indian,"[water, vegetable oil, wheat, salt]",water vegetable oil wheat salt
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",black pepper shallots cornflour cayenne pepper...


In [22]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
vectorized_ingredients = count_vectorizer.fit_transform(train.X)
print('20 features: '+ str(count_vectorizer.get_feature_names()[0:20]))
print('Total features: '+ str(len(count_vectorizer.get_feature_names())))
print(vectorized_ingredients.toarray()[0:10])

vectorized_ingredients_df = pd.DataFrame(columns = count_vectorizer.get_feature_names(), data = vectorized_ingredients.toarray())
vectorized_ingredients_df['num_ingr'] = vectorized_ingredients_df.sum(axis = 1)
vectorized_ingredients_df.head()

20 features: ['00', '10', '100', '14', '15', '25', '33', '40', '43', '95', '96', 'abalone', 'abbamele', 'absinthe', 'abura', 'acai', 'accent', 'accompaniment', 'achiote', 'acid']
Total features: 3010
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


,00,10,100,14,15,25,33,40,43,95,...,zatarain,zatarains,zero,zest,zesty,zinfandel,ziti,zucchini,épices,num_ingr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,16
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33


In [23]:
# encode (label) targets:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
cuisine = label_encoder.fit_transform(train_df['cuisine'].values)
cuisine

array([ 6, 16,  4, ...,  8,  3, 13])

In [24]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
y = cuisine.copy()
X = vectorized_ingredients_df.values
num_selected = 1000
X_count_selected = SelectKBest(chi2, k=num_selected).fit_transform(X, y)
X_count_selected.shape

(39774, 1000)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

tfidf_vectorized_ingredients = tfidf_vectorizer.fit_transform(train.X)
print('20 features: '+ str(tfidf_vectorizer.get_feature_names()[0:20]))
print('Total features: '+ str(len(tfidf_vectorizer.get_feature_names())))
print(tfidf_vectorized_ingredients.toarray()[0:10])

tfidf_vectorized_ingredients_df = pd.DataFrame(columns = tfidf_vectorizer.get_feature_names(), data = tfidf_vectorized_ingredients.toarray())
tfidf_vectorized_ingredients_df['sum'] = tfidf_vectorized_ingredients_df.sum(axis = 1)
tfidf_vectorized_ingredients_df.head()


20 features: ['00', '10', '100', '14', '15', '25', '33', '40', '43', '95', '96', 'abalone', 'abbamele', 'absinthe', 'abura', 'acai', 'accent', 'accompaniment', 'achiote', 'acid']
Total features: 3010
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


,00,10,100,14,15,25,33,40,43,95,...,za,zatarain,zatarains,zero,zest,zesty,zinfandel,ziti,zucchini,épices
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
y = cuisine.copy()
X = tfidf_vectorized_ingredients_df.values.copy()
X_tfidf_selected = SelectKBest(chi2, k=num_selected).fit_transform(X, y)
X_tfidf_selected.shape

(39774, 1000)

In [27]:
# join data
y = cuisine.copy()
print(X_count_selected.shape)
print(X_tfidf_selected.shape)
X = np.concatenate((X_count_selected.copy(),X_tfidf_selected.copy()), axis=1)
X.shape

(39774, 1000)
(39774, 1000)


(39774, 2000)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape)
print(y_train.shape)

(26648, 2000)
(26648,)


In [29]:
import tensorflow as tf
# Imports
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Num GPUs Available:  1
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5



In [30]:
# Imports
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Flatten


# Configuration options
feature_vector_length = X.shape[1]
num_classes = pd.unique(y).shape[0]

X_train = X_train.reshape(X_train.shape[0], feature_vector_length)
X_test = X_test.reshape(X_test.shape[0], feature_vector_length)

# Convert into greyscale
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Convert target classes to categorical ones
Y_train = to_categorical(y_train, num_classes)
Y_test = to_categorical(y_test, num_classes)

# Set the input shape
input_shape = (feature_vector_length,)
print(f'Feature shape: {input_shape}')

# Create the model
model = Sequential(name='embedding_dense_mlpc')
model.add(Embedding(input_dim=len(X.shape), output_dim=20, input_length=feature_vector_length))
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Configure the model and start training
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X_train, Y_train, epochs=10, batch_size=250, verbose=1, validation_split=0.2)
model.summary()

Feature shape: (2000,)
Model: "embedding_dense_mlpc"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2000, 20)          40        
_________________________________________________________________
flatten_3 (Flatten)          (None, 40000)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 300)               12000300  
_________________________________________________________________
dense_13 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_14 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_15 (Dense)             (None, 20)                1020      
Total params: 12,036,510
Trainable params: 12,036,510
Non-trainable params: 0
___________

In [31]:
# model.fit(X_train, Y_train, epochs=int(np.round(X.shape[0]/10)), batch_size=10, verbose=1, validation_split=0.2)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
model.fit(X_train, Y_train, epochs=30, batch_size=1200, verbose=1, validation_split=0.15,callbacks=[callback])
# int(np.round(X.shape[0]/10))
# Test the model after training
test_results = model.evaluate(X_test, Y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

Epoch 1/30
19/19 [==============================] - 3s 129ms/step - loss: 2.6702 - accuracy: 0.1982 - val_loss: 2.5835 - val_accuracy: 0.1628
Epoch 2/30
19/19 [==============================] - 2s 118ms/step - loss: 2.4874 - accuracy: 0.2698 - val_loss: 2.3436 - val_accuracy: 0.3717
Epoch 3/30
19/19 [==============================] - 2s 119ms/step - loss: 2.1096 - accuracy: 0.4295 - val_loss: 1.8671 - val_accuracy: 0.5068
Epoch 4/30
19/19 [==============================] - 2s 116ms/step - loss: 1.7022 - accuracy: 0.5236 - val_loss: 1.5774 - val_accuracy: 0.5493
Epoch 5/30
19/19 [==============================] - 2s 117ms/step - loss: 1.4678 - accuracy: 0.5748 - val_loss: 1.3675 - val_accuracy: 0.5973
Epoch 6/30
19/19 [==============================] - 2s 119ms/step - loss: 1.2673 - accuracy: 0.6324 - val_loss: 1.2435 - val_accuracy: 0.6373
Epoch 7/30
19/19 [==============================] - 2s 117ms/step - loss: 1.1468 - accuracy: 0.6648 - val_loss: 1.1486 - val_accuracy: 0.6651
Epoch 

In [32]:
test_results = model.evaluate(X_test, Y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

411/411 [==============================] - 3s 6ms/step - loss: 0.8154 - accuracy: 0.7618
Test results - Loss: 0.8153969645500183 - Accuracy: 0.7618467211723328%
